# Amazon SageMaker Autopilot Candidate Definition Notebook

This notebook was automatically generated by the AutoML job **niy-Apr-2019-20201202-1434**.
This notebook allows you to customize the candidate definitions and execute the SageMaker Autopilot workflow.

The dataset has **12** columns and the column named **ARR_DELAY_NEW** is used as
the target column. This is being treated as a **Regression** problem. 
This notebook will build a **[Regression](https://en.wikipedia.org/wiki/Regression_analysis)** model that
**minimizes** the "**MSE**" quality metric of the trained models.
The "**MSE**" metric stands for mean square error. It minimizes the square distance between the model's prediction and the true answer.

As part of the AutoML job, the input dataset has been randomly split into two pieces, one for **training** and one for
**validation**. This notebook helps you inspect and modify the data transformation approaches proposed by Amazon SageMaker Autopilot. You can interactively
train the data transformation models and use them to transform the data. Finally, you can execute a multiple algorithm hyperparameter optimization (multi-algo HPO)
job that helps you find the best model for your dataset by jointly optimizing the data transformations and machine learning algorithms.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>
Look for sections like this for recommended settings that you can change.
</div>


---

## Contents

1. [Sagemaker Setup](#Sagemaker-Setup)
    1. [Downloading Generated Candidates](#Downloading-Generated-Modules)
    1. [SageMaker Autopilot Job and Amazon Simple Storage Service (Amazon S3) Configuration](#SageMaker-Autopilot-Job-and-Amazon-Simple-Storage-Service-(Amazon-S3)-Configuration)
1. [Candidate Pipelines](#Candidate-Pipelines)
    1. [Generated Candidates](#Generated-Candidates)
    1. [Selected Candidates](#Selected-Candidates)
1. [Executing the Candidate Pipelines](#Executing-the-Candidate-Pipelines)
    1. [Run Data Transformation Steps](#Run-Data-Transformation-Steps)
    1. [Multi Algorithm Hyperparameter Tuning](#Multi-Algorithm-Hyperparameter-Tuning)
1. [Model Selection and Deployment](#Model-Selection-and-Deployment)
    1. [Tuning Job Result Overview](#Tuning-Job-Result-Overview)
    1. [Model Deployment](#Model-Deployment)

---

## Sagemaker Setup

Before you launch the SageMaker Autopilot jobs, we'll setup the environment for Amazon SageMaker
- Check environment & dependencies.
- Create a few helper objects/function to organize input/output data and SageMaker sessions.

**Minimal Environment Requirements**

- Jupyter: Tested on `JupyterLab 1.0.6`, `jupyter_core 4.5.0` and `IPython 6.4.0`
- Kernel: `conda_python3`
- Dependencies required
  - `sagemaker-python-sdk>=1.72.1,<2.0.0`
    - Use `!pip install sagemaker==1.72.1` to download this dependency.
    - Kernel may need to be restarted after download.
- Expected Execution Role/permission
  - S3 access to the bucket that stores the notebook.

### Downloading Generated Modules
Download the generated data transformation modules and an SageMaker Autopilot helper module used by this notebook.
Those artifacts will be downloaded to **niy-Apr-2019-20201202-1434-artifacts** folder.

In [1]:
!mkdir -p niy-Apr-2019-20201202-1434-artifacts
!aws s3 sync s3://niy-certification/automl/output/niy-Apr-2019-20201202-1434/sagemaker-automl-candidates/pr-1-bcecb56184f44b75be8236d67223d3a14325f9b64ffb4e2c8fa8a5cb91/generated_module niy-Apr-2019-20201202-1434-artifacts/generated_module --only-show-errors
!aws s3 sync s3://niy-certification/automl/output/niy-Apr-2019-20201202-1434/sagemaker-automl-candidates/pr-1-bcecb56184f44b75be8236d67223d3a14325f9b64ffb4e2c8fa8a5cb91/notebooks/sagemaker_automl niy-Apr-2019-20201202-1434-artifacts/sagemaker_automl --only-show-errors

import sys
sys.path.append("niy-Apr-2019-20201202-1434-artifacts")

### SageMaker Autopilot Job and Amazon Simple Storage Service (Amazon S3) Configuration

The following configuration has been derived from the SageMaker Autopilot job. These items configure where this notebook will
look for generated candidates, and where input and output data is stored on Amazon S3.

In [2]:
from sagemaker_automl import uid, AutoMLLocalRunConfig

# Where the preprocessed data from the existing AutoML job is stored
BASE_AUTOML_JOB_NAME = 'niy-Apr-2019-20201202-1434'
BASE_AUTOML_JOB_CONFIG = {
    'automl_job_name': BASE_AUTOML_JOB_NAME,
    'automl_output_s3_base_path': 's3://niy-certification/automl/output/niy-Apr-2019-20201202-1434',
    'data_transformer_image_repo_version': '0.2-1-cpu-py3',
    'algo_image_repo_versions': {'xgboost': '1.0-1-cpu-py3', 'linear-learner': 'latest', 'mlp': 'training-cpu'},
    'algo_inference_image_repo_versions': {'xgboost': '1.0-1-cpu-py3', 'linear-learner': 'latest', 'mlp': 'inference-cpu'}
}

# Path conventions of the output data storage path from the local AutoML job run of this notebook
LOCAL_AUTOML_JOB_NAME = 'niy-Apr-20-notebook-run-{}'.format(uid())
LOCAL_AUTOML_JOB_CONFIG = {
    'local_automl_job_name': LOCAL_AUTOML_JOB_NAME,
    'local_automl_job_output_s3_base_path': 's3://niy-certification/automl/output/niy-Apr-2019-20201202-1434/{}'.format(LOCAL_AUTOML_JOB_NAME),
    'data_processing_model_dir': 'data-processor-models',
    'data_processing_transformed_output_dir': 'transformed-data',
    'multi_algo_tuning_output_dir': 'multi-algo-tuning'
}

AUTOML_LOCAL_RUN_CONFIG = AutoMLLocalRunConfig(
    role='arn:aws:iam::898627427171:role/service-role/AmazonSageMaker-ExecutionRole-20201106T104926',
    base_automl_job_config=BASE_AUTOML_JOB_CONFIG,
    local_automl_job_config=LOCAL_AUTOML_JOB_CONFIG,
    security_config={'EnableInterContainerTrafficEncryption': False, 'VpcConfig': {}})

AUTOML_LOCAL_RUN_CONFIG.display()

2020-12-03 10:54:38,069 INFO botocore.credentials: Found credentials in shared credentials file: ~/.aws/credentials


This notebook is initialized to use the following configuration: 
        <table>
        <tr><th colspan=2>Name</th><th>Value</th></tr>
        <tr><th>General</th><th>Role</th><td>arn:aws:iam::898627427171:role/service-role/AmazonSageMaker-ExecutionRole-20201106T104926</td></tr>
        <tr><th rowspan=2>Base AutoML Job</th><th>Job Name</th><td>niy-Apr-2019-20201202-1434</td></tr>
        <tr><th>Base Output S3 Path</th><td>s3://niy-certification/automl/output/niy-Apr-2019-20201202-1434</td></tr>
        <tr><th rowspan=5>Interactive Job</th><th>Job Name</th><td>niy-Apr-20-notebook-run-03-09-54-38</td></tr>
        <tr><th>Base Output S3 Path</th><td>s3://niy-certification/automl/output/niy-Apr-2019-20201202-1434/niy-Apr-20-notebook-run-03-09-54-38</td></tr>
        <tr><th>Data Processing Trained Model Directory</th><td>s3://niy-certification/automl/output/niy-Apr-2019-20201202-1434/niy-Apr-20-notebook-run-03-09-54-38/data-processor-models</td></tr>
        <tr><th>Data Processing Transformed Output</th><td>s3://niy-certification/automl/output/niy-Apr-2019-20201202-1434/niy-Apr-20-notebook-run-03-09-54-38/transformed-data</td></tr>
        <tr><th>Algo Tuning Model Output Directory</th><td>s3://niy-certification/automl/output/niy-Apr-2019-20201202-1434/niy-Apr-20-notebook-run-03-09-54-38/multi-algo-tuning</td></tr>
        </table>
        

## Candidate Pipelines

The `AutoMLLocalRunner` keeps track of selected candidates and automates many of the steps needed to execute feature engineering and tuning steps.

In [3]:
from sagemaker_automl import AutoMLInteractiveRunner, AutoMLLocalCandidate

automl_interactive_runner = AutoMLInteractiveRunner(AUTOML_LOCAL_RUN_CONFIG)

### Generated Candidates

The SageMaker Autopilot Job has analyzed the dataset and has generated **10** machine learning
pipeline(s) that use **3** algorithm(s). Each pipeline contains a set of feature transformers and an
algorithm.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. The resource configuration: instance type & count
1. Select candidate pipeline definitions by cells
1. The linked data transformation script can be reviewed and updated. Please refer to the [README.md](./niy-Apr-2019-20201202-1434-artifacts/generated_module/README.md) for detailed customization instructions.
</div>

**[dpp0-xgboost](niy-Apr-2019-20201202-1434-artifacts/generated_module/candidate_data_processors/dpp0.py)**: This data transformation strategy first transforms 'numeric1' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical1' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [4]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp0",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": False,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

**[dpp1-xgboost](niy-Apr-2019-20201202-1434-artifacts/generated_module/candidate_data_processors/dpp1.py)**: This data transformation strategy first transforms 'numeric1' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical1' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [5]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp1",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": False,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

**[dpp2-xgboost](niy-Apr-2019-20201202-1434-artifacts/generated_module/candidate_data_processors/dpp2.py)**: This data transformation strategy first transforms 'numeric1' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical1' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [6]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp2",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": False,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

**[dpp3-xgboost](niy-Apr-2019-20201202-1434-artifacts/generated_module/candidate_data_processors/dpp3.py)**: This data transformation strategy first transforms 'numeric1' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical1' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [7]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp3",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": False,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

**[dpp4-linear-learner](niy-Apr-2019-20201202-1434-artifacts/generated_module/candidate_data_processors/dpp4.py)**: This data transformation strategy first transforms 'numeric1' features using [combined RobustImputer and RobustMissingIndicator](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py) followed by [QuantileExtremeValuesTransformer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/base.py), 'categorical1' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustPCA](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/decomposition/robust_pca.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *linear-learner* model. Here is the definition:

In [8]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp4",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": False,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "linear-learner",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

2020-12-03 10:55:00,238 WARNING sagemaker.amazon.amazon_estimator: 'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
2020-12-03 10:55:00,244 WARNING sagemaker.amazon.amazon_estimator: 'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


**[dpp5-linear-learner](niy-Apr-2019-20201202-1434-artifacts/generated_module/candidate_data_processors/dpp5.py)**: This data transformation strategy first transforms 'numeric1' features using [RobustImputer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical1' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *linear-learner* model. Here is the definition:

In [9]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp5",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": False,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "linear-learner",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

2020-12-03 10:55:00,578 WARNING sagemaker.amazon.amazon_estimator: 'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.
2020-12-03 10:55:00,584 WARNING sagemaker.amazon.amazon_estimator: 'get_image_uri' method will be deprecated in favor of 'ImageURIProvider' class in SageMaker Python SDK v2.


**[dpp6-xgboost](niy-Apr-2019-20201202-1434-artifacts/generated_module/candidate_data_processors/dpp6.py)**: This data transformation strategy first transforms 'numeric1' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical1' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [10]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp6",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": False,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

**[dpp7-xgboost](niy-Apr-2019-20201202-1434-artifacts/generated_module/candidate_data_processors/dpp7.py)**: This data transformation strategy first transforms 'numeric1' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical1' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [11]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp7",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": False,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

**[dpp8-xgboost](niy-Apr-2019-20201202-1434-artifacts/generated_module/candidate_data_processors/dpp8.py)**: This data transformation strategy first transforms 'numeric1' features using [RobustImputer (converts missing values to nan)](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py), 'categorical1' features using [ThresholdOneHotEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py). It merges all the generated features and applies [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *xgboost* model. Here is the definition:

In [12]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp8",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": False,
        "transformed_data_format": "application/x-recordio-protobuf",
        "sparse_encoding": True
    },
    "algorithm": {
        "name": "xgboost",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
    }
})

**[dpp9-mlp](niy-Apr-2019-20201202-1434-artifacts/generated_module/candidate_data_processors/dpp9.py)**: This data transformation strategy transforms 'categorical1' features using [ThresholdOrdinalEncoder](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/encoders.py), 'numeric1' features using [RobustImputer](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/impute/base.py) followed by [RobustStandardScaler](https://github.com/aws/sagemaker-scikit-learn-extension/blob/master/src/sagemaker_sklearn_extension/preprocessing/data.py). The
transformed data will be used to tune a *mlp* model. Here is the definition:

In [13]:
automl_interactive_runner.select_candidate({
    "data_transformer": {
        "name": "dpp9",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
            "volume_size_in_gb":  50
        },
        "transform_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "transforms_label": False,
        "transformed_data_format": "text/csv",
        "sparse_encoding": False
    },
    "algorithm": {
        "name": "mlp",
        "training_resource_config": {
            "instance_type": "ml.m5.4xlarge",
            "instance_count": 1,
        },
        "candidate_specific_static_hyperparameters": {
            "num_categorical_features": '5',
        }
    }
})

### Selected Candidates

You have selected the following candidates (please run the cell below and click on the feature transformer links for details):

In [14]:
automl_interactive_runner.display_candidates()

Candidate Name,Algorithm,Feature Transformer
dpp0-xgboost,xgboost,dpp0.py
dpp1-xgboost,xgboost,dpp1.py
dpp2-xgboost,xgboost,dpp2.py
dpp3-xgboost,xgboost,dpp3.py
dpp4-linear-learner,linear-learner,dpp4.py
dpp5-linear-learner,linear-learner,dpp5.py
dpp6-xgboost,xgboost,dpp6.py
dpp7-xgboost,xgboost,dpp7.py
dpp8-xgboost,xgboost,dpp8.py
dpp9-mlp,mlp,dpp9.py


The feature engineering pipeline consists of two SageMaker jobs:

1. Generated trainable data transformer Python modules like [dpp0.py](niy-Apr-2019-20201202-1434-artifacts/generated_module/candidate_data_processors/dpp0.py), which has been downloaded to the local file system
2. A **training** job to train the data transformers
3. A **batch transform** job to apply the trained transformation to the dataset to generate the algorithm compatible data

The transformers and its training pipeline are built using open sourced **[sagemaker-scikit-learn-container][]** and **[sagemaker-scikit-learn-extension][]**.

[sagemaker-scikit-learn-container]: https://github.com/aws/sagemaker-scikit-learn-container
[sagemaker-scikit-learn-extension]: https://github.com/aws/sagemaker-scikit-learn-extension

## Executing the Candidate Pipelines

Each candidate pipeline consists of two steps, feature transformation and algorithm training.
For efficiency first execute the feature transformation step which will generate a featurized dataset on S3
for each pipeline.

After each featurized dataset is prepared, execute a multi-algorithm tuning job that will run tuning jobs
in parallel for each pipeline. This tuning job will execute training jobs to find the best set of
hyper-parameters for each pipeline, as well as finding the overall best performing pipeline.

### Run Data Transformation Steps

Now you are ready to start execution all data transformation steps.  The cell below may take some time to finish,
feel free to go grab a cup of coffee. To expedite the process you can set the number of `parallel_jobs` to be up to 10.
Please check the account limits to increase the limits before increasing the number of jobs to run in parallel.

In [15]:
automl_interactive_runner.fit_data_transformers(parallel_jobs=2)

2020-12-03 10:55:25,667 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 10:55:25,668 WARNING sagemaker: This is not the latest supported version. If you would like to use version 0.23-1, please add framework_version=0.23-1 to your constructor.
2020-12-03 10:55:25,669 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 10:55:25,669 INFO root: [Worker_0:dpp0-xgboost]Executing step: train_data_transformer
2020-12-03 10:55:25,670 WARNING sagemaker: This is not the latest supported version. If you would like to use version 0.23-1, please add framework_version=0.23-1 to your constructor.
2020-12-03 10:55:25,672 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 10:55:25,672 WARNING sagemaker: This is not the latest supported version. If you would like to use version 0.23-1, please add framewo

### Multi Algorithm Hyperparameter Tuning

Now that the algorithm compatible transformed datasets are ready, you can start the multi-algorithm model tuning job
to find the best predictive model. The following algorithm training job configuration for each
algorithm is auto-generated by the AutoML Job as part of the recommendation.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. Hyperparameter ranges
2. Objective metrics
3. Recommended static algorithm hyperparameters.

Please refers to [Xgboost tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost-tuning.html) and [Linear learner tuning](https://docs.aws.amazon.com/sagemaker/latest/dg/linear-learner-tuning.html) for detailed explanations of the parameters.
</div>

The AutoML recommendation job has recommended the following hyperparameters, objectives and accuracy metrics for
the algorithm and problem type:

In [16]:
ALGORITHM_OBJECTIVE_METRICS = {
    'xgboost': 'validation:mse',
    'linear-learner': 'validation:objective_loss',
    'mlp': 'validation:mse',
}

STATIC_HYPERPARAMETERS = {
    'xgboost': {
        'objective': 'reg:squarederror',
        'save_model_on_termination': 'true',
    },
    'linear-learner': {
        'predictor_type': 'regressor',
        'epochs': 50,
        'loss': 'auto',
        'mini_batch_size': 800,
    },
    'mlp': {
        'problem_type': 'regression',
        'ml_application': 'mlp',
        'use_batchnorm': 'true',
        'activation': 'relu',
        'warmup_epochs': 10,
        'eval_metric': 'mse',
    },
}

The following tunable hyperparameters search ranges are recommended for the Multi-Algo tuning job:

In [17]:
from sagemaker.parameter import CategoricalParameter, ContinuousParameter, IntegerParameter

ALGORITHM_TUNABLE_HYPERPARAMETER_RANGES = {
    'xgboost': {
        'num_round': IntegerParameter(2, 1024, scaling_type='Logarithmic'),
        'max_depth': IntegerParameter(2, 8, scaling_type='Logarithmic'),
        'eta': ContinuousParameter(1e-3, 1.0, scaling_type='Logarithmic'),
        'gamma': ContinuousParameter(1e-6, 64.0, scaling_type='Logarithmic'),
        'min_child_weight': ContinuousParameter(1e-6, 32.0, scaling_type='Logarithmic'),
        'subsample': ContinuousParameter(0.5, 1.0, scaling_type='Linear'),
        'colsample_bytree': ContinuousParameter(0.3, 1.0, scaling_type='Linear'),
        'lambda': ContinuousParameter(1e-6, 2.0, scaling_type='Logarithmic'),
        'alpha': ContinuousParameter(1e-6, 2.0, scaling_type='Logarithmic'),
    },
    'linear-learner': {
        'wd': ContinuousParameter(1e-7, 1.0, scaling_type='Logarithmic'),
        'l1': ContinuousParameter(1e-7, 1.0, scaling_type='Logarithmic'),
        'learning_rate': ContinuousParameter(1e-5, 1.0, scaling_type='Logarithmic'),
    },
    'mlp': {
        'mini_batch_size': IntegerParameter(128, 512, scaling_type='Linear'),
        'learning_rate': ContinuousParameter(1e-6, 1e-2, scaling_type='Logarithmic'),
        'weight_decay': ContinuousParameter(1e-12, 1e-2, scaling_type='Logarithmic'),
        'dropout_prob': ContinuousParameter(0.25, 0.5, scaling_type='Linear'),
        'embedding_size_factor': ContinuousParameter(0.65, 0.95, scaling_type='Linear'),
        'network_type': CategoricalParameter(['feedforward', 'widedeep']),
        'layers': CategoricalParameter(['256', '50, 25', '100, 50', '200, 100', '256, 128', '300, 150', '200, 100, 50']),
    },
}

#### Prepare Multi-Algorithm Tuner Input

To use the multi-algorithm HPO tuner, prepare some inputs and parameters. Prepare a dictionary whose key is the name of the trained pipeline candidates and the values are respectively:

1. Estimators for the recommended algorithm
2. Hyperparameters search ranges
3. Objective metrics

In [18]:
multi_algo_tuning_parameters = automl_interactive_runner.prepare_multi_algo_parameters(
    objective_metrics=ALGORITHM_OBJECTIVE_METRICS,
    static_hyperparameters=STATIC_HYPERPARAMETERS,
    hyperparameters_search_ranges=ALGORITHM_TUNABLE_HYPERPARAMETER_RANGES)

2020-12-03 11:37:35,225 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 11:37:35,226 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 11:37:35,226 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 11:37:35,227 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 11:37:35,227 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 11:37:35,228 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 11:37:35,228 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 11:37:35,229 WARNING sagemaker.estimator: Parameter image_name will be r

Below you prepare the inputs data to the multi-algo tuner:

In [19]:
multi_algo_tuning_inputs = automl_interactive_runner.prepare_multi_algo_inputs()

2020-12-03 11:37:40,350 WARNING sagemaker: 's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
2020-12-03 11:37:40,351 WARNING sagemaker: 's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
2020-12-03 11:37:40,352 WARNING sagemaker: 's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
2020-12-03 11:37:40,352 WARNING sagemaker: 's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
2020-12-03 11:37:40,353 WARNING sagemaker: 's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
2020-12-03 11:37:40,353 WARNING sagemaker: 's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
2020-12-03 11:37:40,354 WARNING sagemaker: 's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
2020-12-03 11:37:40,354 WARNING sagemaker: 's3_input' class will be renamed to 'TrainingInput' in SageMaker Python SDK v2.
2020-12-03 11:37

#### Create Multi-Algorithm Tuner

With the recommended Hyperparameter ranges and the transformed dataset, create a multi-algorithm model tuning job
that coordinates hyper parameter optimizations across the different possible algorithms and feature processing strategies.

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. Tuner strategy: [Bayesian](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Bayesian_optimization), [Random Search](https://en.wikipedia.org/wiki/Hyperparameter_optimization#Random_search)
2. Objective type: `Minimize`, `Maximize`, see [optimization](https://en.wikipedia.org/wiki/Mathematical_optimization)
3. Max Job size: the max number of training jobs HPO would be launching to run experiments. Note the default value is **250**
    which is the default of the managed flow.
4. Parallelism. Number of jobs that will be executed in parallel. Higher value will expedite the tuning process.
    Please check the account limits to increase the limits before increasing the number of jobs to run in parallel
5. Please use a different tuning job name if you re-run this cell after applied customizations.
</div>

In [21]:
from sagemaker.tuner import HyperparameterTuner

base_tuning_job_name = "{}-tuning".format(AUTOML_LOCAL_RUN_CONFIG.local_automl_job_name)

tuner = HyperparameterTuner.create(
    base_tuning_job_name=base_tuning_job_name,
    strategy='Bayesian',
    objective_type='Minimize',
    max_parallel_jobs=2,
    max_jobs=100,
    **multi_algo_tuning_parameters,
)

#### Run Multi-Algorithm Tuning

Now you are ready to start running the **Multi-Algo Tuning** job. After the job is finished, store the tuning job name which you use to select models in the next section.
The tuning process will take some time, please track the progress in the Amazon SageMaker Hyperparameter tuning jobs console.

In [22]:
from IPython.display import display, Markdown

# Run tuning
tuner.fit(inputs=multi_algo_tuning_inputs, include_cls_metadata=None)
tuning_job_name = tuner.latest_tuning_job.name

display(
    Markdown(f"Tuning Job {tuning_job_name} started, please track the progress from [here](https://{AUTOML_LOCAL_RUN_CONFIG.region}.console.aws.amazon.com/sagemaker/home?region={AUTOML_LOCAL_RUN_CONFIG.region}#/hyper-tuning-jobs/{tuning_job_name})"))

# Wait for tuning job to finish
tuner.wait()

2020-12-03 11:38:23,075 INFO sagemaker.tuner: _TuningJob.start_new!!!
2020-12-03 11:38:23,078 INFO sagemaker: Creating hyperparameter tuning job with name: niy-Apr-20-notebook--201203-1138


Tuning Job niy-Apr-20-notebook--201203-1138 started, please track the progress from [here](https://eu-central-1.console.aws.amazon.com/sagemaker/home?region=eu-central-1#/hyper-tuning-jobs/niy-Apr-20-notebook--201203-1138)

........................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................................

## Model Selection and Deployment

This section guides you through the model selection process. Afterward, you construct an inference pipeline
on Amazon SageMaker to host the best candidate.

Because you executed the feature transformation and algorithm training in two separate steps, you now need to manually
link each trained model with the feature transformer that it is associated with. When running a regular Amazon
SageMaker Autopilot job, this will automatically be done for you.

### Tuning Job Result Overview

The performance of each candidate pipeline can be viewed as a Pandas dataframe. For more interactive usage please
refers to [model tuning monitor](https://docs.aws.amazon.com/sagemaker/latest/dg/automatic-model-tuning-monitor.html).

In [23]:
from pprint import pprint
from sagemaker.analytics import HyperparameterTuningJobAnalytics

SAGEMAKER_SESSION = AUTOML_LOCAL_RUN_CONFIG.sagemaker_session
SAGEMAKER_ROLE = AUTOML_LOCAL_RUN_CONFIG.role

tuner_analytics = HyperparameterTuningJobAnalytics(
    tuner.latest_tuning_job.name, sagemaker_session=SAGEMAKER_SESSION)

df_tuning_job_analytics = tuner_analytics.dataframe()

# Sort the tuning job analytics by the final metrics value
df_tuning_job_analytics.sort_values(
    by=['FinalObjectiveValue'],
    inplace=True,
    ascending=False if tuner.objective_type == "Maximize" else True)

# Show detailed analytics for the top 20 models
df_tuning_job_analytics.head(20)

,l1,learning_rate,wd,TrainingJobName,TrainingJobStatus,FinalObjectiveValue,TrainingStartTime,TrainingEndTime,TrainingElapsedTimeSeconds,TrainingJobDefinitionName,...,max_depth,min_child_weight,num_round,subsample,dropout_prob,embedding_size_factor,layers,mini_batch_size,network_type,weight_decay
23,NaN,NaN,NaN,niy-Apr-20-notebook--201203-1138-077-ed92cba9,Completed,112.364372,2020-12-03 14:41:35+01:00,2020-12-03 14:43:58+01:00,143.0,dpp6-xgboost,...,6.0,0.000246,1019.0,0.841792,NaN,NaN,NaN,NaN,NaN,NaN
31,NaN,NaN,NaN,niy-Apr-20-notebook--201203-1138-069-4a564742,Completed,115.137993,2020-12-03 14:18:07+01:00,2020-12-03 14:20:12+01:00,125.0,dpp6-xgboost,...,8.0,0.000002,681.0,0.771873,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,NaN,NaN,niy-Apr-20-notebook--201203-1138-094-40cddec1,Completed,116.878860,2020-12-03 15:15:53+01:00,2020-12-03 15:16:58+01:00,65.0,dpp6-xgboost,...,7.0,4.515284,222.0,0.982084,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,niy-Apr-20-notebook--201203-1138-099-52485a29,Completed,118.013763,2020-12-03 15:27:50+01:00,2020-12-03 15:29:54+01:00,124.0,dpp6-xgboost,...,6.0,0.000001,916.0,0.959909,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,niy-Apr-20-notebook--201203-1138-098-4b1595c6,Completed,119.172188,2020-12-03 15:23:30+01:00,2020-12-03 15:26:01+01:00,151.0,dpp6-xgboost,...,8.0,0.000025,919.0,0.785819,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,NaN,NaN,niy-Apr-20-notebook--201203-1138-091-fdcad7ba,Completed,121.680878,2020-12-03 15:11:29+01:00,2020-12-03 15:13:34+01:00,125.0,dpp6-xgboost,...,8.0,1.570702,753.0,0.878672,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,NaN,NaN,niy-Apr-20-notebook--201203-1138-093-622bb41a,Completed,124.040466,2020-12-03 15:15:36+01:00,2020-12-03 15:17:27+01:00,111.0,dpp6-xgboost,...,6.0,0.000004,758.0,0.687855,NaN,NaN,NaN,NaN,NaN,NaN
18,NaN,NaN,NaN,niy-Apr-20-notebook--201203-1138-082-fc5f9c06,Completed,124.662079,2020-12-03 14:51:22+01:00,2020-12-03 14:52:54+01:00,92.0,dpp6-xgboost,...,5.0,0.000410,605.0,0.762083,NaN,NaN,NaN,NaN,NaN,NaN
34,NaN,NaN,NaN,niy-Apr-20-notebook--201203-1138-066-474c7ea8,Completed,127.815331,2020-12-03 14:13:32+01:00,2020-12-03 14:14:43+01:00,71.0,dpp3-xgboost,...,6.0,3.125752,255.0,0.567103,NaN,NaN,NaN,NaN,NaN,NaN
13,NaN,NaN,NaN,niy-Apr-20-notebook--201203-1138-087-7d0c8742,Completed,128.604858,2020-12-03 15:03:09+01:00,2020-12-03 15:05:28+01:00,139.0,dpp6-xgboost,...,7.0,0.000001,946.0,0.662445,NaN,NaN,NaN,NaN,NaN,NaN


The best training job can be selected as below:

<div class="alert alert-info"> 💡 <strong>Tips: </strong>
You could select alternative job by using the value from `TrainingJobName` column above and assign to `best_training_job` below
</div>

In [24]:
attached_tuner = HyperparameterTuner.attach(tuner.latest_tuning_job.name, sagemaker_session=SAGEMAKER_SESSION)
best_training_job = attached_tuner.best_training_job()

print("Best Multi Algorithm HPO training job name is {}".format(best_training_job))

2020-12-03 15:32:21,495 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 15:32:21,545 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 15:32:21,546 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 15:32:21,548 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 15:32:21,567 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 15:32:21,568 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 15:32:21,569 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 15:32:21,572 WARNING sagemaker.estimator: Parameter image_name will be r

### Linking Best Training Job with Feature Pipelines

Finally, deploy the best training job to Amazon SageMaker along with its companion feature engineering models.
At the end of the section, you get an endpoint that's ready to serve online inference or start batch transform jobs!

Deploy a [PipelineModel](https://sagemaker.readthedocs.io/en/stable/pipeline.html) that has multiple containers of the following:

1. Data Transformation Container: a container built from the model we selected and trained during the data transformer sections
2. Algorithm Container: a container built from the trained model we selected above from the best HPO training job.

Get both best data transformation model and algorithm model from best training job and create an pipeline model:

In [25]:
from sagemaker.estimator import Estimator
from sagemaker import PipelineModel
from sagemaker_automl import select_inference_output

# Get a data transformation model from chosen candidate
best_candidate = automl_interactive_runner.choose_candidate(df_tuning_job_analytics, best_training_job)
best_data_transformer_model = best_candidate.get_data_transformer_model(role=SAGEMAKER_ROLE, sagemaker_session=SAGEMAKER_SESSION)

# Our first data transformation container will always return recordio-protobuf format
best_data_transformer_model.env["SAGEMAKER_DEFAULT_INVOCATIONS_ACCEPT"] = 'application/x-recordio-protobuf'
# Add environment variable for sparse encoding
if best_candidate.data_transformer_step.sparse_encoding:
    best_data_transformer_model.env["AUTOML_SPARSE_ENCODE_RECORDIO_PROTOBUF"] = '1'

# Get a algo model from chosen training job of the candidate
algo_estimator = Estimator.attach(best_training_job)
best_algo_model = algo_estimator.create_model(**best_candidate.algo_step.get_inference_container_config())

# Final pipeline model is composed of data transformation models and algo model and an
# inverse label transform model if we need to transform the intermediates back to non-numerical value
model_containers = [best_data_transformer_model, best_algo_model]
if best_candidate.transforms_label:
    model_containers.append(best_candidate.get_data_transformer_model(
        transform_mode="inverse-label-transform",
        role=SAGEMAKER_ROLE,
        sagemaker_session=SAGEMAKER_SESSION))


pipeline_model = PipelineModel(
    name="AutoML-{}".format(AUTOML_LOCAL_RUN_CONFIG.local_automl_job_name),
    role=SAGEMAKER_ROLE,
    models=model_containers,
    vpc_config=AUTOML_LOCAL_RUN_CONFIG.vpc_config)

2020-12-03 15:33:20,952 INFO root: Chosen Data Processing pipeline candidate name is dpp6-xgboost
2020-12-03 15:33:21,387 WARNING sagemaker.estimator: Parameter image_name will be renamed to image_uri in SageMaker Python SDK v2.
2020-12-03 15:33:21,392 WARNING sagemaker: This is not the latest supported version. If you would like to use version 0.23-1, please add framework_version=0.23-1 to your constructor.
2020-12-03 10:23:10 Starting - Preparing the instances for training
2020-12-03 10:23:10 Downloading - Downloading input data
2020-12-03 10:23:10 Training - Training image download completed. Training in progress.
2020-12-03 10:23:10 Uploading - Uploading generated training model
2020-12-03 10:23:10 Completed - Training job completed2020-12-03 10:22:38,052 sagemaker-training-toolkit INFO     Imported framework sagemaker_sklearn_container.training
2020-12-03 10:22:38,053 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2020-12-03 10:22:38,062 sagemak

### Deploying Best Pipeline

<div class="alert alert-info"> 💡 <strong> Available Knobs</strong>

1. You can customize the initial instance count and instance type used to deploy this model.
2. Endpoint name can be changed to avoid conflict with existing endpoints.

</div>

Finally, deploy the model to SageMaker to make it functional.

In [ ]:
pipeline_model.deploy(initial_instance_count=1,
                      instance_type='ml.m5.2xlarge',
                      endpoint_name=pipeline_model.name,
                      wait=True)

Congratulations! Now you could visit the sagemaker
[endpoint console page](https://eu-central-1.console.aws.amazon.com/sagemaker/home?region=eu-central-1#/endpoints) to find the deployed endpoint (it'll take a few minutes to be in service).

<div class="alert alert-warning">
    <strong>To rerun this notebook, delete or change the name of your endpoint!</strong> <br>
If you rerun this notebook, you'll run into an error on the last step because the endpoint already exists. You can either delete the endpoint from the endpoint console page or you can change the <code>endpoint_name</code> in the previous code block.
</div>